In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 2.5MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad

In [2]:
import sys
sys.path.insert(0,'/usr/local/bin/chromedriver')
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import json
import pprint
import time


In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [4]:
def open_and_cokie(url_link):
  link_ristoranti = []
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  wd.get(url_link)
  try:
    button = wd.find_element_by_css_selector('body > div:nth-child(28) > div > div > div > button')
    button.click()
    button = wd.find_element_by_css_selector('#cookie-banner > div > div > div > span > button > span')
    button.click()
  except:
    pass
  try:
    time.sleep(10)
    list_raw = wd.find_elements_by_css_selector("#__next > div > div > div.ccl-d523c73794f30c03.ccl-917a4d5b8cd3a9e3.ccl-71fbf9dc5f85ebd4 > div > div.HomeLayout-b45e85df675abd0e > div > div > p ")
    numero_ristoranti = int(list_raw[0].text.split(' ')[0])
    for i in range(1,numero_ristoranti):
      link_ristoranti.append(wd.find_element_by_xpath(f"//*[@id=\"__next\"]/div/div/div[2]/div/div[2]/div/ul/li[{i}]/div/div/a").get_attribute('href'))
  except:
    pass
  return(link_ristoranti)

In [ ]:
def scraping_delivero(link_risto):
  info_ristoranti = []
  info_menu = []
  list_image = []
  try:
      for _ in range(len(link_risto)):
        time.sleep(3)
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        wd = webdriver.Chrome('chromedriver',options=chrome_options)
        wd.get(link_risto[_])
        print(wd.title)
        Query_Area_Nome_ristorante = wd.find_elements_by_css_selector("li.orderweb__ac637a69")
        query_ranking = wd.find_elements_by_css_selector(".orderweb__61671603")
        Macroarea = Query_Area_Nome_ristorante[1].text
        Nome_ristorante = Query_Area_Nome_ristorante[2].text
        Descrizione = wd.find_elements_by_css_selector("#app-element > div > div > div > div > div > div.restaurant--main > div > div > ul")
        Indirizzo = Descrizione[0].text.split("\n")[-2]
        Orario_apertura = Descrizione[0].text.split("\n")[-3]
        keyword_ristorante = Descrizione[0].text.split("\n")[:-3]
        Categorie = wd.find_elements_by_css_selector("#app-element > div > div > div > div > div > div > h3") # categorie 
        ranking = query_ranking[0].text
        info_ristoranti.append({'url': link_risto[_],
                              'title': Nome_ristorante,
                              'macro_area': Macroarea,
                              'indirizzo':Indirizzo,
                              'keyword_ristorante':keyword_ristorante,
                              'orario_apertura':Orario_apertura,
                              'ranking':ranking.split("(")[0],
                              'total_rating':ranking.split("(")[-1].split("valutazioni")[0]})
        for i in range(len(Categorie)):
          item = wd.find_elements_by_css_selector(f"#app-element > div > div > div > div > div:nth-child({i+2}) > ul > li > div") # item
          if len(item) >0 :
            for y in range(len(item)):
              try: 
                if item[y].text.split("\n")[0] !='':
                  if item[y].text.split("\n")[-1] == "Popolare":
                    info_menu.append({'url': link_risto[_],
                            'title': Nome_ristorante,
                            'categorie':Categorie[i].text,
                            'item':item[y].text.split("\n"),
                            'prezzo':item[y].text.split("\n")[-2],
                            'popolare':True})
                  else:
                    info_menu.append({'url': link_risto[_],
                            'title': Nome_ristorante,
                            'categorie':Categorie[i].text,
                            'item':item[y].text.split("\n"),
                            'prezzo':item[y].text.split("\n")[-1],
                            'popolare':False})
                else:
                  continue
              except:
                continue
        else:
          continue
  except:
    pass  
  return(info_ristoranti,info_menu)

In [7]:
if __name__ == "__main__":
    lnk_rest = open_and_cokie("https://deliveroo.it/en/restaurants/milano/ripamonti?geohash=u0nd8be8xmf0&sort=rating")
    print(len(lnk_rest))
    info_rist,info_food = scraping_delivero(lnk_rest)
    import pandas as pd
    ds_info_ristoranti = pd.DataFrame(info_rist)
    ds_info_food = pd.DataFrame(info_food)
    print(ds_info_ristoranti.shape)
    print(ds_info_food.shape)
    ds_info_ristoranti.to_csv("ds_info_ristoranti_delivero.tsv",sep="\t",header=True,index=False)
    ds_info_food.to_csv("ds_info_food_delivero.tsv",sep="\t",header=True,index=False)

375
Consegna a domicilio di Bar D’Este a Porta Romana - Ordina con Deliveroo
Consegna a domicilio di Panarello a Missori - Ordina con Deliveroo
Consegna a domicilio di Ziva Pasticceria a De Angeli - Ravizza - Ordina con Deliveroo
Consegna a domicilio di Starbucks a Missori - Ordina con Deliveroo
Consegna a domicilio di Pasticceria Clivati a Tortona - Ordina con Deliveroo
Consegna a domicilio di Pasticceria Viscontea a San Vittore - Ordina con Deliveroo
Consegna a domicilio di Kate Away a Ticinese - Ordina con Deliveroo
Consegna a domicilio di Cremeria Buonarroti a De Angeli - Ravizza - Ordina con Deliveroo
Consegna a domicilio di Gelati Clivati a Tortona - Ordina con Deliveroo
Consegna a domicilio di Venchi a Magenta - Ordina con Deliveroo
Consegna a domicilio di Calibreakfast by Poke House - Organic sweet bowls & more a Guastalla - Ordina con Deliveroo
Consegna a domicilio di Dolci Memela a Barona - Ordina con Deliveroo
Consegna a domicilio di Verdi's Breakfast & Bistrot a Sant'Ambrog

In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('drive')
!cp ds_info_ristoranti.tsv "drive/My Drive/"
!cp ds_info_food.tsv "drive/My Drive/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [6]:
def scraping_delivero(link_risto):
  info_ristoranti = []
  info_menu = []
  list_image = []
  for _ in range(len(link_risto)):
    try:
      time.sleep(3)
      chrome_options = webdriver.ChromeOptions()
      chrome_options.add_argument('--headless')
      chrome_options.add_argument('--no-sandbox')
      chrome_options.add_argument('--disable-dev-shm-usage')
      wd = webdriver.Chrome('chromedriver',options=chrome_options)
      wd.get(link_risto[_])
      print(wd.title)
      Query_Area_Nome_ristorante = wd.find_elements_by_css_selector("li.orderweb__ac637a69")
      query_ranking = wd.find_elements_by_css_selector(".orderweb__61671603")
      Macroarea = Query_Area_Nome_ristorante[1].text
      Nome_ristorante = Query_Area_Nome_ristorante[2].text
      Descrizione = wd.find_elements_by_css_selector("#app-element > div > div > div > div > div > div.restaurant--main > div > div > ul")
      Indirizzo = Descrizione[0].text.split("\n")[-2]
      Orario_apertura = Descrizione[0].text.split("\n")[-3]
      keyword_ristorante = Descrizione[0].text.split("\n")[:-3]
      Categorie = wd.find_elements_by_css_selector("#app-element > div > div > div > div > div > div > h3") # categorie 
      ranking = query_ranking[0].text
      info_ristoranti.append({'url': link_risto[_],
                            'title': Nome_ristorante,
                            'macro_area': Macroarea,
                            'indirizzo':Indirizzo,
                            'keyword_ristorante':keyword_ristorante,
                            'orario_apertura':Orario_apertura,
                            'ranking':ranking.split("(")[0],
                            'total_rating':ranking.split("(")[-1].split("valutazioni")[0]})
      for i in range(len(Categorie)):
        item = wd.find_elements_by_css_selector(f"#app-element > div > div > div > div > div:nth-child({i+2}) > ul > li > div") # item
        #print(item[0].text)
        info_menu.append({'url': link_risto[_],
                        'title': Nome_ristorante,
                        'categorie':Categorie[i].text,
                        'item':item[0].text.split("\n")})
      #print(info_menu)
    except:
      continue  
  return(info_ristoranti,info_menu)